In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import copy 

import seaborn as sns
sns.set(color_codes=True)

In [2]:
from sklearn.model_selection import train_test_split

def split_dataset(data, n:float, seed=0):
    dummy_labels = np.ones(data.shape[0])
    X_train, X_test, y_train, y_test = train_test_split(data, dummy_labels, test_size=n, random_state=seed)
    return [X_train, X_test]


In [3]:
my_data = pd.read_csv('../data/DS_1.csv', header=None) .values

In [4]:
gan_data = pd.read_csv('../data/gan_dirichlet_1.csv', header=None) .values

In [5]:
train_set, test_set = split_dataset(my_data, n=0.2)

# Error de Validacion

En esta parte hay que definir un error para nuestra GAN.

## Clasificacion 
+ Para problemas de **clasificacion** una medida del error puede ser **clasificaciones correctas/numero total de clasificacones**
+ Para problemas de clasificacion multi clase existe una perdida logaritmica definida como: $$\frac{-1}{N} \mathop{\sum_{i=1}^{N}\sum_{j=1}^{M}} y_{ij}*log(p_{ij})$$

Donde:
+ $y_{ij}$ indica si la muestra i pertenece a la clase j
+ $p_{ij}$  es la probabiliad de que la muestra i pertenezca a la calse j 

+ **F1 Score** : $$2 \times \frac{1}{ \frac{1}{precision} + \frac{1}{recall}}$$. Precision y recall se definen en terminos de TruePositives, FalsePositives y FalseNegatives

## Regresion

+ Mean Absolute Error
+ Mean Squared Error

## Medidas para las GANS

En el caso de las GANs, la funcion objetivo o de perdida mide que tan bien el generador engaña al discriminador. Entonces, esta no nos dice mucho acerca de que tan reaslaticas o diversas son las muestras generadas por la GAN.

Para el caso de **GAN de imagenes**, hay dos medidas del rendimiento de una gan, el Inception Score y el Frechet Inception Distance. (https://medium.com/@jonathan_hui/gan-how-to-measure-gan-performance-64b988c47732)

**Una buena GAN es una que genera muestras diversas y realistas**. En el caso de las imagenes, no es trivial hacer que una computadora te diga que tan realistas es una imagen.

# Adapted Inception Score

In [6]:
def DKL(pk, qk):
    return scipy.stats.entropy(pk,qk)      